In [345]:
import ops
from ops.imports_ipython import *

# runs example from repository directory
home = os.path.dirname(os.path.dirname(ops.__file__))
os.chdir(os.path.join(home, 'projects', 'steph'))
print(os.getcwd())

/Users/sasha/PycharmProjects/OpticalPooledScreens/projects/steph


/Users/sasha/PycharmProjects/OpticalPooledScreens/venv/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [346]:
WILDCARDS = dict(well=3, tile=0) # change these to change the well and tile that you want to analyze
CYCLES = 11 # number of cycles

THRESHOLD_READS = 50
THRESHOLD_DAPI = 2000
THRESHOLD_CELL = 2500
NUCLEUS_AREA = 40, 400

# color of bases
# lut = "lookup table", used to map one color to another like a filter
LUTS = [
    ops.io.GRAY,
    ops.io.GREEN,
    ops.io.RED,
    ops.io.MAGENTA,
    ops.io.CYAN
]

# for formatting tif images when they are saved?
DISPLAY_RANGES = [
    [500, 15000],
    [100, 10000],
    [100, 20000],
    [100, 8000],
    [100, 6000]
]

In [347]:
barcodes = pd.read_csv('design.csv') # list of barcodes along with which gene they target
barcodes["prefix"] = barcodes["barcode"].apply(lambda x: x[:CYCLES])
barcode_set = set(barcodes["prefix"])

In [348]:
# find sbs images and print paths
search = f'data/10x_Cycle*_Well{WILDCARDS["well"]}_Point3_{str(WILDCARDS["tile"]).rjust(4, "0")}*.ome.tif'
input_files = natsorted(glob(search))
print(len(input_files))
print(input_files)
# used to format output filenames
description = {'mag': "10X", "well": WILDCARDS["well"], 'tile': WILDCARDS['tile'], 'subdir': f'process_ipynb/tile{WILDCARDS["tile"]}', 'ext': 'tif'}

11
['data/10x_Cycle1_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0784.ome.tif', 'data/10x_Cycle2_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0784.ome.tif', 'data/10x_Cycle3_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0784.ome.tif', 'data/10x_Cycle4_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0784.ome.tif', 'data/10x_Cycle5_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0784.ome.tif', 'data/10x_Cycle6_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0784.ome.tif', 'data/10x_Cycle7_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0784.ome.tif', 'data/10x_Cycle8_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0392.ome.tif', 'data/10x_Cycle9_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0784.ome.tif', 'data/10x_Cycle10_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0784.ome.tif', 'data/10x_Cycle11_Well3_Point3_0000_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0784.ome.tif']


In [349]:
data = np.array([read(f) for f in input_files])

In [350]:
data, x_offsets, y_offsets = Snake.align_SBS(data, method="SBS_mean") # rigid alignment of sequencing cycles and channels.
# save(name(description, tag='aligned'), data, display_ranges=DISPLAY_RANGES, luts=LUTS)

In [351]:
loged = Snake.transform_log(data, skip_index=0) # apply Laplacian-of-Gaussian filter from scipy.ndimage.
# save(name(description, tag='log'), loged, display_ranges=DISPLAY_RANGES, luts=LUTS)

In [352]:
maxed = Snake.max_filter(loged, 3, remove_index=0) # apply a maximum filter in a window of `width`. Conventionally operates on Laplacian-of-Gaussian filtered SBS data, dilating sequencing channels to compensate for single-pixel alignment error.
# save(name(description, tag='maxed'), maxed, display_ranges=DISPLAY_RANGES[1:], luts=LUTS[1:])

[autoreload of ops.firesnake failed: Traceback (most recent call last):
  File "/Users/sasha/PycharmProjects/OpticalPooledScreens/venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/sasha/PycharmProjects/OpticalPooledScreens/venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/sasha/PycharmProjects/OpticalPooledScreens/venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/Users/sasha/PycharmProjects/OpticalPooledScreens/venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 329, in update_class
    if update_generic(old_obj, new_obj):
  File "/Users/sasha/PycharmProjects/OpticalPooledScreens/venv/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/Users/

In [353]:
std = Snake.compute_std(loged, remove_index=0) # use standard deviation over cycles, followed by mean across channels to estimate sequencing read locations.
# save(name(description, tag='std'), std)

In [354]:
peaks = Snake.find_peaks(std) # where are the spots
# save(name(description, tag='peaks'), peaks)

### segment nuclei and cells

In [355]:
# Find nuclei from DAPI (fluorescent stain)
# change first argument if DAPI staining is only done for a certain cycle, eg. 11th cycle would be data[10]
nuclei = Snake.segment_nuclei(data[0], THRESHOLD_DAPI,
 area_min=NUCLEUS_AREA[0], area_max=NUCLEUS_AREA[1])

save(name(description, tag='nuclei'), nuclei, compress=1)

In [356]:
cells = Snake.segment_cells(data[0], nuclei, THRESHOLD_CELL) # Matches cell labels to nuclei labels.
save(name(description, tag='cells'), cells, compress=1)

### extract base intensity, call reads, assign to cells

In [357]:
# Find the signal intensity from `maxed` at each point in `peaks` above `threshold_peaks`.
df_bases = Snake.extract_bases(maxed, peaks, cells,
                        THRESHOLD_READS, wildcards=WILDCARDS)
# df_bases.to_csv(name(description, tag='bases', ext='csv'), index=None)

In [358]:
df_reads = Snake.call_reads(df_bases, peaks=peaks) # call reads by compensating for channel cross-talk and calling the base with the highest corrected intensity for each cycle. Q = quality?
filename = name(description, tag='reads', ext='csv')
df_reads.to_csv(filename, index=None)

In [359]:
# read from csv to match numerical precision of snakemake pipeline
df_reads = pd.read_csv(name(description, tag='reads', ext='csv'))
df_cells = Snake.call_cells(df_reads, df_pool=barcodes) # gets the two most-common barcode reads for each cell (prioritizes barcodes found in gene pool)
df_cells.to_csv(name(description, tag='cells', ext='csv'), index=None)

In [360]:
if WILDCARDS["tile"] not in pd.read_csv("process_ipynb/cells.csv")["tile"]:
    df_cells.to_csv("process_ipynb/cells.csv", mode="a", index=False, header=False)

### annotated SBS images

In [361]:
# last channel annotates base calls
annotate_luts = LUTS + [ops.annotate.GRMC, ops.io.GRAY]
annotate_display_ranges = [(a / 4, b / 4) for a,b in DISPLAY_RANGES] + [(0, 4)]
annotate_SBS = Snake.annotate_SBS(log=loged, df_reads=df_reads)

save(name(description, tag='annotate_SBS'), annotate_SBS,
     display_ranges=annotate_display_ranges, luts=annotate_luts, compress=1)

### idk :)

In [362]:
from collections import defaultdict

In [363]:
barcode_counts = defaultdict(int)

for index, row in df_cells.iterrows():
    b0 = row["barcode_0"]
    b1 = row["barcode_1"]
    if b0 in barcode_set: # lots of messed up reads...
        barcode_counts[b0] += int(row["count_0"])
    if b1 in barcode_set:
        barcode_counts[b1] += int(float(row["count_1"]))

print(sorted(barcode_counts.items(), key=lambda x: x[1], reverse=True))

[('ATAATAGCATT', 18), ('CTCAACCACTT', 14), ('GACAGACTGGT', 9), ('GAAGAAGGATG', 9), ('AGGTCTGCCCA', 9), ('CTCTCAGACCT', 8), ('ACTACTAGCTC', 7), ('TCTAGCTGAGT', 7), ('AAAGCTATGTT', 6), ('GTTGCCAGCTT', 6), ('GCACTATTAAT', 5), ('CTGATCAGGAT', 5), ('ACCATCGCATG', 4), ('GCGATGGCCTC', 4), ('CGAAAGATACC', 4), ('TGGACAGTGCC', 4), ('GGAATGGGAAG', 4), ('TTGCCTCACCC', 4), ('CTCCCCGACTA', 3), ('ACTCTCTAATA', 3), ('GATCGCTTTCC', 3), ('CCAACAACGTA', 3), ('CTCAAACTTAT', 3), ('CCCACCACCCC', 3), ('ATGAGTCAAAG', 3), ('TCTTTCCAACT', 3), ('AGCCCCTCCTT', 3), ('GGTCGTGGGTC', 2), ('AGGTGGGGCTG', 2), ('AAGTTCTTGAA', 2), ('GTATGGGCTGC', 2), ('GTGCCACAAAC', 2), ('GGTCCCGCAGC', 2), ('CCAGCTTTGAT', 2), ('CGCCTGCGCAC', 2), ('TTCTCTATGTA', 2), ('AGATGCGAGCT', 2), ('GATCTCAACTC', 2), ('GGTGAAGTCGT', 2), ('GTATATCCTCT', 2), ('CCTCTCCCAAT', 2), ('TTAGCCGGATG', 2), ('AACCGAGAGGG', 1), ('ACTACAGTCCA', 1), ('ACCCGACAAAT', 1), ('TGTCGACAAAT', 1), ('ACGCAGGTTAA', 1), ('TGAGCTTCTGG', 1), ('GGTCCCCAACC', 1), ('GTTTCGAAACT', 1